In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import ast

In [3]:
# Loading data into pandas dataframe
path = "../../data/clean/"
file = "merged_agg_session.csv"
cols = ['Port ID', 'Interval Energy', 'start_seconds', 'start_year', 'start_month', 'start_day']

In [4]:
df = pd.read_csv(path + file, index_col=False, usecols=cols)

In [5]:
def seconds_to_96_steps(x):
    return x // 900

In [ ]:
def apply_inplace(df, field, fun):
    return pd.concat([df.drop(field, axis=1), df[field].apply(fun)], axis=1) 

In [6]:
df['step'] = df['start_seconds'].apply(seconds_to_96_steps)
# df.drop(columns=['start_seconds'], inplace=True)
# df.drop(columns=['start_month'], inplace=True)

In [23]:
# print(df['start_year'].value_counts())

In [24]:
# print(df['Port ID'].value_counts().head(10))
# print(np.percentile(df['Port ID'].value_counts(), 20))
# print(np.percentile(df['Port ID'].value_counts(), 50))
# print(np.percentile(df['Port ID'].value_counts(), 80))

In [38]:
df = df[~df['Port ID'].isnull()]
df['Port ID'] = df['Port ID'].apply(lambda x: str(int(x)))
ports = df['Port ID'].value_counts()
# print(ports)
print(len(ports))
ports = ports[ports >= 100]
print(len(ports))
ports = [str(int(x)) for x in ports.index.values]

15682
12549


In [39]:
df = df[df['Port ID'].isin(ports)]

# Create df for ports, fully filled in time

In [30]:
# total_steps = 96 * (4*365 + 1) # 4 years + leap year
# print(total_steps)

140256


In [68]:
# not feasible, will take forever.

def get_df_for_year(df, year):
    df_year = df[df['start_year'] == year]
    print("apply str to list transform")
    df_year = apply_inplace(df_year, 'Interval Energy', ast.literal_eval)
    max_idx = 96*365 if year != 2016 else 96*366
    data_year = pd.DataFrame(0, index=np.arange(max_idx), columns=ports)
#     print(data_year.head(10))
    num_rows = len(df_year)
    i = 0
    for index, row in df_year.iterrows(): 
        i += 1
        print(i, " / ", num_rows)
        energy = row['Interval Energy']
#         energy = ast.literal_eval(row['Interval Energy'])
#         energy = np.array(ast.literal_eval(row['Interval Energy']))
        start_idx = (row['start_day'] - 1)*96 + row['step']
        end_idx = start_idx + len(energy) - 1 # note; pandas includes end index
        if end_idx > max_idx:
            end_idx = max_idx
            energy = energy[:(end_idx - start_idx)]
            # print
#             print(end_idx - start_idx, len(energy))
#             print(start_idx, end_idx, max_idx, energy)
#             print(len(data_year.loc[start_idx:end_idx, row['Port ID']]))
#             print(data_year.loc[start_idx:end_idx, row['Port ID']])
        data_year.loc[start_idx:end_idx, row['Port ID']] = energy

In [69]:
df_2015 = get_df_for_year(df, year=2015)

/home/ec2-user/anaconda3/envs/eda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


1  /  1298745
2  /  1298745
3  /  1298745
4  /  1298745
5  /  1298745
6  /  1298745
7  /  1298745
8  /  1298745
9  /  1298745
10  /  1298745
11  /  1298745
12  /  1298745
13  /  1298745
14  /  1298745
15  /  1298745
16  /  1298745
17  /  1298745
18  /  1298745
19  /  1298745
20  /  1298745
21  /  1298745
22  /  1298745
23  /  1298745
24  /  1298745
25  /  1298745
26  /  1298745
27  /  1298745
28  /  1298745
29  /  1298745
30  /  1298745
31  /  1298745
32  /  1298745
33  /  1298745
34  /  1298745
35  /  1298745
36  /  1298745
37  /  1298745
38  /  1298745
39  /  1298745
40  /  1298745


ValueError: Must have equal len keys and value when setting with an iterable